Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` removing 'raise NotImplementedError()'. Make usre you also fill your student id below:

---

## Part A: Regular expressions (2 points)

In [2]:
import re
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.corpus import gutenberg

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Complete the following function using the search function of the re library to find and return the year that a file in the gutenberg collection was written. Return the year as string or the empty string if not found.

In [22]:
def get_year(file):
    # raw text
    text_raw = gutenberg.raw(file)

    # The publication years are between 1000 and 2024
    match = re.findall(r'\b1\d{3}|20[0-1]\d{1}|202[0-4]\b', text_raw)

    # Return the first match found
    if match:
        return match[0]
    # Return an empty string if no match is found
    return ""

In [6]:
"""Testing that the function returns the correct results"""
assert get_year('carroll-alice.txt') == '1865'
assert get_year('shakespeare-hamlet.txt') == '1599'

In [23]:
get_year('carroll-alice.txt')

'1865'

Complete thw following function using the findall function of the re library to compute and report the number of times some text appears in a file of the gutenberg collection.

In [9]:
def find_mentions(text, file):
    # raw text
    text_raw = gutenberg.raw(file)

    # Count the text appearing in the file of the guternberg
    match = re.findall(text, text_raw)

    # Return the count of text
    return len(match)

In [10]:
"""Testing that the function returns the correct results"""
assert find_mentions('CHAPTER ', 'carroll-alice.txt') == 12
assert find_mentions('UNK', 'shakespeare-caesar.txt') == 0

Complete the following function to return the mean number of words in the sentences of a gutenberg file that is given as input. The number should be rounded to 2 decimals.

In [25]:
def average_sentence_length(file):
    # have the sentences
    sentences = gutenberg.sents(file)

    # have the words
    words = gutenberg.words(file)

    # Calculate the mean number of words in the sentences of a gutenberg file
    sent_length = round(len(words)/len(sentences),2)

    # Return the result
    return sent_length

In [26]:
"""Testing that the function returns the correct results"""
assert average_sentence_length('shakespeare-caesar.txt') == 11.94
assert average_sentence_length('austen-emma.txt') == 24.82

## Part B: Byte-Pair Encoding (4 points)

Implement a function that returns the vocabulary learned by the byte-pair encoding algorithm given a corpus, after a number of iterations. The characters in the corpus, sorted in alphabetical order, will be the first members of the vocabulary, to be extended with as many tokens as the number of iterations given.

In [27]:
from collections import Counter

def bpe_train(text, num_iters):
# Tokenize the corpus, marking the end of each word with "_"
    words = [word + "_" for word in corpus.split()]

    # Initialize vocabulary with all unique characters in the corpus, sorted alphabetically
    vocab = sorted(set("".join(words)))

    # Convert the words into lists of characters to facilitate merging
    words = [list(word) for word in words]

    # Run BPE for the specified number of iterations
    for _ in range(num_iters):
        # Count frequency of each adjacent pair in the corpus
        pairs = Counter()
        for word in words:
            for i in range(len(word) - 1):
                pair = (word[i], word[i + 1])
                pairs[pair] += 1

        # If no pairs are left to merge, break early
        if not pairs:
            break

        # Find the most frequent pair
        most_frequent_pair = max(pairs, key=pairs.get)
        new_token = "".join(most_frequent_pair)
        vocab.append(new_token)

        # Merge the most frequent pair in each word
        new_words = []
        for word in words:
            new_word = []
            i = 0
            while i < len(word):
                # Check if we can merge the current pair
                if i < len(word) - 1 and (word[i], word[i + 1]) == most_frequent_pair:
                    new_word.append(new_token)
                    i += 2  # Skip over the pair
                else:
                    new_word.append(word[i])
                    i += 1
            new_words.append(new_word)
        words = new_words

    return vocab

In [28]:
"""Testing that the function returns the correct results"""
corpus = "low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new"
assert set(bpe_train(corpus, 8)) == set(['_', 'd', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', 'er', 'er_', 'ne', 'new', 'lo', 'low', 'newer_', 'low_']
)
corpus = "hello world my large language model is coming after you beware"
assert set(bpe_train(corpus, 5)) == set(['_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'r', 's', 't', 'u', 'w', 'y', 'e_', 'el', 'la', 'ge_', 'ng']
)

## Part C: Minimum Edit Distance (4 points)

Extend the code of the minimum edit distance algorithm to implement the following function that returns a list of at least one alignment. Each alignment should be a list containing the source string with stars for insertion, the target string with stars for deletion, and the path of operations consisting of characters s, i, d and - for substitution, insertion, deletion, and nothing respectively. See the testing cell for examples.

In [38]:
import numpy as np


def compute_alignments(seq1, seq2, substitution_cost=1):
    m, n = len(seq1), len(seq2)

    # Initialize DP and backtrack tables using numpy
    dp = np.zeros((m + 1, n + 1), dtype=int)
    backtrack = np.full((m + 1, n + 1), None)

    # Fill initial costs for converting prefixes
    dp[:, 0] = np.arange(m + 1)    # Cost of deletions
    dp[0, :] = np.arange(n + 1)    # Cost of insertions
    backtrack[1:, 0] = 'd'         # Mark deletions in backtrack
    backtrack[0, 1:] = 'i'         # Mark insertions in backtrack

    # Fill DP and backtrack arrays
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            cost_substitute = dp[i - 1, j - 1] + (substitution_cost if seq1[i - 1] != seq2[j - 1] else 0)
            cost_insert = dp[i, j - 1] + 1
            cost_delete = dp[i - 1, j] + 1

            # Select minimum cost and operation
            min_cost = min(cost_substitute, cost_insert, cost_delete)
            dp[i, j] = min_cost
            if min_cost == cost_substitute:
                backtrack[i, j] = 's' if seq1[i - 1] != seq2[j - 1] else '-'
            elif min_cost == cost_insert:
                backtrack[i, j] = 'i'
            else:
                backtrack[i, j] = 'd'

    # Backtracking to get one or more optimal alignments
    def backtrack_alignments(i, j):
        if i == 0 and j == 0:
            return [("", "", "")]  # Base case: empty alignment

        alignments = []
        if i > 0 and backtrack[i, j] == 'd':
            for prev_seq1, prev_seq2, prev_ops in backtrack_alignments(i - 1, j):
                alignments.append((prev_seq1 + seq1[i - 1], prev_seq2 + '*', prev_ops + 'd'))

        if j > 0 and backtrack[i, j] == 'i':
            for prev_seq1, prev_seq2, prev_ops in backtrack_alignments(i, j - 1):
                alignments.append((prev_seq1 + '*', prev_seq2 + seq2[j - 1], prev_ops + 'i'))

        if i > 0 and j > 0:
            if backtrack[i, j] == 's':
                for prev_seq1, prev_seq2, prev_ops in backtrack_alignments(i - 1, j - 1):
                    alignments.append((prev_seq1 + seq1[i - 1], prev_seq2 + seq2[j - 1], prev_ops + 's'))
            elif backtrack[i, j] == '-':
                for prev_seq1, prev_seq2, prev_ops in backtrack_alignments(i - 1, j - 1):
                    alignments.append((prev_seq1 + seq1[i - 1], prev_seq2 + seq2[j - 1], prev_ops + '-'))

        return alignments

    # Get the alignments starting from (m, n)
    all_alignments = backtrack_alignments(m, n)

    # Format alignments for output
    formatted_alignments = []
    for seq1_aligned, seq2_aligned, ops in all_alignments:
        formatted_alignments.append([seq1_aligned, seq2_aligned, ops])

    return formatted_alignments

In [39]:
"""Testing that the function returns the correct results"""
for alignment in compute_alignments('nuts', 'bolts', 1):
    assert alignment in [['*nuts', 'bolts', 'iss--'], ['n*uts', 'bolts', 'sis--'], ['nu*ts', 'bolts', 'ssi--']]
for alignment in compute_alignments('ab', 'db', 2):
    assert alignment in [['ab', 'db', 's-'], ['*ab', 'd*b', 'id-'], ['a*b', '*db', 'di-']]